# _`ETL`_

Import the libraries

In [1]:
import pandas as pd
import ast

Import the dataset as a DataFrame and storage it in the data variable

In [2]:
data = pd.read_csv('_src/data/movies_dataset.csv', encoding='UTF-8', delimiter=',', )

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14980\593806153.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('_src/data/movies_dataset.csv', encoding='UTF-8', delimiter=',', )


Import the credits dataset for later on extract the director name

In [3]:
credits = pd.read_csv('_src/data/credits.csv')
credits.drop(columns=['cast'], inplace=True)

Transform id into an integer

In [4]:
try:
    data['id'].astype(int)
except ValueError as e:
    print(e)

invalid literal for int() with base 10: '1997-08-20'


We can see that the data is all wrong, so we drop it

In [5]:
data.loc[data['id'].str.contains('-', regex=True)]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.drop(data.loc[data['id'].str.contains('-', regex=True)].index, inplace=True)

In [7]:
data['id'] = data['id'].astype(int)

Merge the dataframes so we have a dataframe with all the data

In [8]:
data = pd.merge(data,credits, on='id')

Drop the innecesary columns

In [9]:
data.drop(columns=['adult','homepage','imdb_id','original_title','poster_path','video','spoken_languages'], inplace = True)

### Extract the nested data that we need and create its dataframe's columns ### 

### Nested data in 'belongs_to_collection'

data: {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

target: collection_name

In [10]:
#Crate the new columns of the DataFrame
data['collection_name'] = ''

#Asign to the new columns the values in belongs_to_collecition < id and name
for i in range(len(data['belongs_to_collection'])):
    if (len(str(data['belongs_to_collection'][i])) > 10):
        data['belongs_to_collection'][i] = data['belongs_to_collection'][i].split(',')
        data.loc[i, 'collection_name'] = data['belongs_to_collection'][i][1][10:-1]

#Drop belongs_to_collection
data.drop(columns = 'belongs_to_collection', inplace = True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14980\3481308185.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['belongs_to_collection'][i] = data['belongs_to_collection'][i].split(',')


### nested data in 'crew'

data : {'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

target : director. In the case that there is more than one, we extract all in a list format




In [11]:
def extract_directors(crew_str):
    member_dict = ast.literal_eval(crew_str)
    directors = [member_dict[i]['name'] for i in range(len(member_dict)) if member_dict[i]['job'] == 'Director']
    return ','.join(directors)

data['directors'] = data['crew'].apply(extract_directors)

### Nested data in 'genres'

data : [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
 
target : genre. In the case that there is more than one, we extract all in a list format


In [12]:
# Define a function to extract genre names from genres
def extract_genre_names(genres_str):
    if(len(str(genres_str)) > 4):
        genre_list = genres_str.split(',')
        genre_names = [element.strip(']')[10:-2] for element in genre_list if element[2:6] == 'name']
        return ','.join(genre_names)

# Apply the function to create the 'genres_list' column
data['genres_list'] = data['genres'].apply(extract_genre_names)

# Drop 'genres' column
data.drop(columns='genres', inplace=True)

### Nested data in 'production_companies'

In genres we keep only the genre's names as a list.

data : [{'name': 'Pixar Animation Studios', 'id': 3}]

target : prod_companies. In case that there is more than one, we extract all in a list format 

In [13]:
#Crate the new columns of the DataFrame
data['prod_companies'] = ''

def extract_comp_names(comp_str):
    if(len(str(comp_str)) > 4):
        genre_list = comp_str.split(',')
        comp_names = [element.strip(']')[11:-1] for element in genre_list if element[3:7] == 'name']
        return ','.join(comp_names)

# Apply the function to create the 'genres_list' column
data['prod_companies'] = data['production_companies'].apply(extract_comp_names)

# Drop 'genres' column
data.drop(columns='production_companies', inplace = True)

data['prod_companies'].fillna('None', inplace = True)

### Nested data in 'production_contries'

In prod_companies we keep only the companies names as a list.

data : [{'iso_3166_1': 'US', 'name': 'United States of America'}]

target : prod_countries. In case that there is more than one, we extract all in a list format 

In [14]:
def extract_countries_names(country_str):
    if(len(str(country_str)) > 4):
        # countries_list = country_str.str.strip('([])')
        countries_list = country_str.split(',')
        countries_names = [element.strip("]")[10:-2] for element in countries_list if element[2:6] == 'name']
        return ','.join(countries_names)

data['prod_countries'] = data['production_countries'].apply(extract_countries_names)

data.drop(columns = 'production_countries', inplace = True)

## Create a column 'return' with the return of investmen. (revenue / budget). In case that there is no data to operate it has to take the value 0

### Before creating the column we have to make sure that the data is in the correct format to make the operations

Fill the nulls values in budget and revenue with 0

In [15]:
fill_values = {'revenue': 0, 'budget': 0}
data.fillna(fill_values, inplace=True)

Now that we have all the data, we change the dtypes of the necesary columns to make the operations

In [16]:
int_columns = ['budget','revenue']
float_columns = ['popularity','vote_average']

data[int_columns] = data[int_columns].astype(int)
data[float_columns] = data[float_columns].astype(float)

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


#### Now we can create the 'return' column

In [17]:
data['return'] = round(data['revenue'] / data['budget'],2)

We have some inf values because there is a revenue and no budget (cero division). So I choose to keep the return as the revenue of the movie

In [18]:
data.loc[data['return'] == float('inf'), 'return'] = data['revenue']

The dates has to have the format AAAA-mm-dd, and with that dates, create the column release_year where we will extract the year of release

In [19]:
data['release_date'] = pd.to_datetime(data['release_date'])
data.dropna(subset=['release_date'], inplace=True)
data['release_year'] = data['release_date'].dt.year

ReIndex the columns for a better order

In [20]:
new_columns = ['collection_name','directors', 'genres_list', 'id', 
               'title', 'prod_companies', 'prod_countries', 'original_language', 
               'overview', 'popularity', 'release_date','release_year', 'runtime', 
               'status', 'tagline', 'vote_average', 'vote_count', 'budget', 'revenue','return']
data = data.reindex(columns = new_columns)

In [21]:
data.loc[data['prod_companies'].isna(), 'title']

Series([], Name: title, dtype: object)

Drop the movies with no release_date

In [22]:
data.drop(data.loc[data['release_date'].isna()].index, inplace=True)

Export the transformed data

In [23]:
data.to_csv('_src/data/movies_transformed.csv', index=False)